<a href="https://colab.research.google.com/github/iued-uni-heidelberg/DAAD-Training-2021/blob/main/sentiment_analysis_lexicons_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment analysis lexicons using Twitter feeds

In [1]:
!wget https://heibox.uni-heidelberg.de/f/da61b08e7ec945e9b2a5/?dl=1

--2022-02-21 11:26:04--  https://heibox.uni-heidelberg.de/f/da61b08e7ec945e9b2a5/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/b64feebb-e3ff-4b58-babf-766867cfcf3f/lexicon-overlap-score-examples-master.tar.gz [following]
--2022-02-21 11:26:05--  https://heibox.uni-heidelberg.de/seafhttp/files/b64feebb-e3ff-4b58-babf-766867cfcf3f/lexicon-overlap-score-examples-master.tar.gz
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 14278919 (14M) [application/octet-stream]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>]  13.62M  8.48MB/s    in 1.6s    

2022-02-21 11:26:07 (8.48 MB/s) - ‘index.html?dl=1’ saved [14278919/14278919]



In [ ]:
!mv index.html?dl=1 lexicon-overlap-score-examples-master.tar.gz
!tar -xvzf lexicon-overlap-score-examples-master.tar.gz

In [ ]:
# -*- coding: utf-8 -*-
# Python script to open each file, read json input and copy to one text file for subsequent processing
import os, re, sys

class clLex2Dict(object):
    '''
    @author Bogdan Babych, IÜD, Heidelberg University, 2021
    @email bogdan [dot] babych [at] iued [dot] uni-heidelberg [dot] de
    '''
    def __init__(self, SDirName, output_file = 'lexicon-tweets-all.txt', filtr = 0): # initialising by openning the directories
        self.SOutput_file = output_file
        self.Filter = filtr
        self.openDir(SDirName)
        return

    def openDir(self, path): # implementation of recursively openning directories from a given rule directory and reading each file recursively into a string
        i = 0
        DLexiconAll = {}
        FOut = open(self.SOutput_file, 'w')

        for root,d_names,f_names in os.walk(path):
            for f in f_names: # for each file found in the directory
                ## remove this if using on another corpus

                fullpath = os.path.join(root, f)
                i+=1
                if i%1==0: 
                    print(str(i) + '. Processing: ' + f)
                    print(fullpath)

                DLexicon1File = self.procFile(fullpath, f, i) # returns converted string + tags
                
                for key, value in DLexicon1File.items():
                    try:
                        LValues = DLexiconAll[key]
                    except:
                        LValues = []
                    '''
                    '''
                    LValues.append(value)
                    DLexiconAll[key] = LValues
        DLexiconMerged, DLexiconMerged2Conts = self.mergeLex(DLexiconAll)
        self.printDict(DLexiconMerged, DLexiconMerged2Conts, FOut)

        # if SText2Write: FOut.write(SText2Write) # if the string is not empty then write to file
        # FIn.close()

        '''
        try:
            pass
        except:
            print(f'file {f} cannot be read or processed')
        finally:
            pass
        '''
        
        FOut.flush()
        FOut.close()

        return


    def procFile(self, fullpath, SFNameIn, i): # sending each json string for extraction of text and attaching an correct tags to each output string output string
        DLexicon1File = {}
        FIn = open(fullpath, 'r')
        for SLine in FIn:
            SLine = SLine.strip()
            try: 
                LLine = re.split('\t', SLine)
                SWord = LLine[0]
                SSentiment = LLine[1]
                ASentiment = float(SSentiment)
                DLexicon1File[SWord] = ASentiment
            except:
                pass

        return DLexicon1File



    def mergeLex(self, DLexiconAll):
        '''

        '''
        DLexiconMerged = {}
        DLexiconMerged2Conts = {}

        
        for key, value in DLexiconAll.items():
            sum = 0
            for el in value:
                try:
                    sum = sum + el
                except:
                    pass
            average = sum / len(value)
            DLexiconMerged[key] = average
            DLexiconMerged2Conts[key] = len(value)


        return DLexiconMerged, DLexiconMerged2Conts

    def printDict(self, Dict, DictConts, FOut):
        for key, value in sorted(Dict.items() , key=lambda x: x[1], reverse=False):
            counts = DictConts[key]
            if self.Filter:
                if counts > self.Filter:
                    FOut.write(key + '\t' + str(value) + '\t' + str(counts) + '\n')
            else:
                FOut.write(key + '\t' + str(value) + '\t' + str(counts) + '\n')
        

# calling the class
OLexicon = clLex2Dict('/content/lexicon-overlap-score-examples-master/lexicons/', filtr = 0)

# Another positive-negative lexicon
extracting from annotated file, putting into tsv

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/9072fdbed717466db460/?dl=1
!mv index.html?dl=1 subjectivity_clues_hltemnlp05.zip
!unzip subjectivity_clues_hltemnlp05.zip

In [48]:
FInLex = open('/content/lexicon-tweets-all.txt', 'r')
# global DTwitterLex
def readTweeterLex(FInLex):
    DTwitterLex = {}
    for SLine in FInLex:
        SLine = SLine.strip()
        LLine = re.split('\t', SLine)
        try:
            SWord = LLine[0]
            SWeight = LLine[1]
            DTwitterLex[SWord] = float(SWeight)
        except:
            print('.')
    return DTwitterLex



In [49]:
SFIn = '/content/subjectivity_clues_hltemnlp05/subjclueslen1-HLTEMNLP05.tff'
SFOut = 'lexicon-positive-negative.txt'
PositiveC = 0
PositiveS = 0

NegativeC = 0
NegativeS = 0

NeutralC = 0
NeutralS = 0

NonFoundC = 0

DTwitterLex = readTweeterLex(FInLex)
print(len(DTwitterLex.items()))
FIn = open(SFIn, 'r')
FOut = open(SFOut, 'w')
for SLine in FIn:
    SLine = SLine.strip()
    LLine = re.split('[ =]', SLine)
    # try:
    SWord = LLine[5]
    SPoS = LLine[7]
    SSentiment = LLine[11]


    # Weight = 'No'
    
    try:
        Weight = DTwitterLex[SWord]
    except:
        Weight = 'NF'

    S2Print = '\t'.join([SWord, SPoS, SSentiment, str(Weight)])
    # except:
    #     pass
    if Weight == 'NF':
        NonFoundC += 1
    elif SSentiment == 'negative':
        NegativeC +=1
        NegativeS += Weight
    elif SSentiment == 'positive':
        PositiveC += 1
        PositiveS += Weight

    elif SSentiment == 'neutral':
        NeutralC +=1
        NeutralS += Weight
       
    FOut.write(S2Print + '\n')
FOut.flush()
FOut.close()

PositiveA = PositiveS/PositiveC
NegativeA = NegativeS/NegativeC
NeutralA = NeutralS/NeutralC

print(PositiveC)
print(PositiveS)
print(PositiveA)

print(NegativeC)
print(NegativeS)
print(NegativeA)

print(NeutralC)
print(NeutralS)
print(NeutralA)

print(NonFoundC)


66092
1807
701.7783006824241
0.38836651947007417
2669
-1438.5379946979776
-0.5389801403889013
469
-48.10320581794548
-0.10256547082717586
3262
